In [3]:
tennis_data <- read_csv(url("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2023.csv"))
tail(tennis_data)

Rows: 2986 Columns: 49
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): tourney_id, tourney_name, surface, tourney_level, winner_entry, wi...
dbl (35): draw_size, tourney_date, match_num, winner_id, winner_seed, winner...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,⋯,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2023-M-DC-2023-WG2-PO-RSA-LUX-01,Davis Cup WG2 PO: RSA vs LUX,NA,4,D,20230204,4,200270,NA,NA,⋯,NA,NA,NA,NA,NA,NA,608,48,516,67
2023-M-DC-2023-WG2-PO-RSA-LUX-01,Davis Cup WG2 PO: RSA vs LUX,NA,4,D,20230204,5,202335,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,1717,1
2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NA,4,D,20230203,1,117365,NA,NA,⋯,NA,NA,NA,NA,NA,NA,990,11,279,190
2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NA,4,D,20230203,2,121411,NA,NA,⋯,NA,NA,NA,NA,NA,NA,364,131,894,15
2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NA,4,D,20230203,4,144949,NA,NA,⋯,NA,NA,NA,NA,NA,NA,894,15,285,184
2023-M-DC-2023-WG2-PO-TUN-CYP-01,Davis Cup WG2 PO: TUN vs CYP,NA,4,D,20230203,5,121411,NA,NA,⋯,NA,NA,NA,NA,NA,NA,364,131,990,11


In [22]:
temp <- tempfile()
download_file <- download.file("https://archive.ics.uci.edu/static/public/186/wine+quality.zip", temp)
red_wine <- read.csv2(unz(temp, "winequality-red.csv"))
white_wine <- read.csv2(unz(temp, "winequality-white.csv"))
unlink(temp)